In [168]:
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np

In [169]:
RANDOM_STATE = 42

In [170]:
dataset = load_boston()
X = pd.DataFrame(dataset.data)
X.columns = dataset.feature_names
y = dataset.target

C:\Users\Хит\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np

        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the Ca

In [171]:
X.shape

(506, 13)

In [172]:
y.shape

(506,)

1. Разделите выборку на обучающую и тестовую в отношении 80%/20%

In [173]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    train_size=0.8,
                                                    random_state=RANDOM_STATE)

In [174]:
X_train.shape

(404, 13)

In [175]:
X_test.shape

(102, 13)

In [176]:
y_train.shape

(404,)

In [177]:
y_test.shape

(102,)

2. Обучите стандартную регрессию, а также Ridge и  Lasso и параметрами по умолчанию и выведите их R2 на тестовой выборке

In [228]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score

In [241]:
def r2_test(model):
    reg = model.fit(X_train, y_train)
    r2 = reg.score(X_test, y_test)
    print('%.7f' % r2)
    try:
        coef = reg.alpha_
        print(f'coef: {coef}')
    except AttributeError:
        print('')

In [242]:
r2_test(LinearRegression())

0.6687595



In [243]:
r2_test(Ridge())

0.6662222



In [244]:
r2_test(Lasso())

0.6671454



3. Для Ridge и Lasso подберите коэффициент регуляризации(используйте GridSearchCV, RidgeCV, LassoCV) в пределах от $10^{-5}$ до $10^5$ (по степеням 10). Посчитайте R2 на тестовой выборке по лучшим моделям и сравните с предыдущими результатами. Напишите как изменился результат

In [253]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import RidgeCV, LassoCV

param_grid = [10**i for i in range(-5,6)]
param_grid

[1e-05, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000, 100000]

In [374]:
def r2_grid(model, param_grid):
    grid_search = GridSearchCV(model, {'alpha' : param_grid}, cv=10)
    grid_search.fit(X_train, y_train)
    r2 = grid_search.score(X_test, y_test)
    coef = grid_search.best_params_['alpha']
    print('%.7f' % r2)
    print (f'coef: {coef}')

In [375]:
r2_grid(Ridge(), param_grid)

0.6687595
coef: 1e-05


In [376]:
r2_grid(Lasso(), param_grid)

0.6687599
coef: 1e-05


In [377]:
r2_test(RidgeCV(param_grid))

0.6687510
coef: 0.01


In [378]:
r2_test(LassoCV(alphas=param_grid))

0.6687599
coef: 1e-05


Результат немного улучшился.

4. Проведите масштабирование выборки(используйте Pipeline, StandardScaler, ), посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

In [296]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

In [297]:
def r2_scale(scaler, reg):
    pipe = Pipeline(steps=[('scaler', scaler), ('reg', reg)]).fit(X_train, y_train)
    r2 = pipe.score(X_test, y_test)
    print('%.7f' % r2)

In [298]:
r2_scale(StandardScaler(), Ridge())

0.6684624


In [299]:
r2_scale(MinMaxScaler(), Ridge())

0.6764100


In [300]:
r2_scale(StandardScaler(), Lasso())

0.6239429


In [301]:
r2_scale(MinMaxScaler(), Lasso())

0.2573921


In [ ]:
Результаты хуже предыдущих, особенно MinMaxScaler() + Lasso()

5. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

In [369]:
def r2_scale_grid(scaler, reg, param_grid):
    pipe = Pipeline(steps=[('scaler', scaler), ('reg', reg)])
    grid_search = GridSearchCV(pipe, {'reg__alpha' : param_grid}, cv=10)
    grid_search.fit(X_train, y_train)
    r2 = grid_search.score(X_test, y_test)
    coef = grid_search.best_params_['reg__alpha']
    print('%.7f' % r2)
    print (f'coef: {coef}')

In [370]:
r2_scale_grid(StandardScaler(), Ridge(), param_grid)

0.6659678
coef: 10


In [371]:
r2_scale_grid(MinMaxScaler(), Ridge(), param_grid)

0.6764100
coef: 1


In [372]:
r2_scale_grid(StandardScaler(), Lasso(), param_grid)

0.6681816
coef: 0.01


In [373]:
r2_scale_grid(MinMaxScaler(), Lasso(), param_grid)

0.6676993
coef: 0.01


Ridge() + StandardScaler() результат немного уменьшился. Ridge() + MinMaxScaler() не изменился. Lasso() + MinMaxScaler() немного увеличился. Lasso() + MinMaxScaler() стал значительно лучше.

6. Добавьте попарные произведения признаков и их квадраты (используйте PolynomialFeatures) на масштабированных признаках, посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

7. Подберите наилучшую модель (используйте Pipeline, GridSearchSCV) подбирая тип регуляризации (L1,L2), коэффициент регуляризации, метод масштабирования и степень полинома в PolynomialFeatures. Выведите итоговые параметры и результат R2. Напишите как изменился R2 по сравнению с предыдущими экспериментами

http://archive.ics.uci.edu/ml/datasets/Adult

In [ ]:
link = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/adult-all.csv'
data = pd.read_csv(link, header=None)

In [ ]:
data.head()

8. Разделите выборку на признаки и целевую переменную(колонка со зачениями {<=50K,>50K}). Замените целевую переменную на числовые значения.

9. Выясните, присутствуют ли в данных пропуски. Заполните их самыми частыми значениями (испольуйте SimpleImputer)

10. Выберите колонки с числовыми и категориальными переменными.

11. Создайте пайплайн по обработке колонок(используйте OneHotEncoder,MinMaxScaler).

12. Посчитайте метрики accuracy и f1_score на предсказании только самого частого класса в целевой переменной.

13. Посчитайте cross_val_score по алгоритмам LogisticRegression, SVC, LinearSVC по метрикам accuracy и f1_score.
Напишите удалось ли превзойти предыдущий результат.

14. Можно заметить что в данных присутствуют значения '?', замените их самыми частыми значениями (испольуйте SimpleImputer)

15. Посчитайте cross_val_score на новых данных. Напишите удалось ли улучшить результат.

16. Посчитайте cross_val_score, если просто удалить значения '?'. Напишите как изменился результат

 17. Посчитайте cross_val_score для RandomForestClassifier,GradientBoostingClassifier. Напишите как изменился результат и какой вывод можно из этого сделать.

18. Подберите наилучшую модель, подбирая методы обработки колонок - масштабирование признаков, кодирование признаков и заполнение пропусков. Параметры алгоритмов оставьте по умолчанию. Выведите итоговые параметры и результат accuracy и f1_score.